In [15]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

#mplleaftlet?

In [16]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_substations = pd.read_excel('./substations.xlsx', index_col=0)

In [17]:
display(df_grid)
display(df_substations)

,name,eic,tso,substation_1,substation_2,voltage,resistance,reactance,susceptance,length,zone
0,Eula - Röhrsdorf 203,11TD8L203------C,50HERTZ,744,570,220,4.0937,18.349,161.163703,51.511,DE
1,Röhrsdorf - Weida 207,11TD8L207------H,50HERTZ,570,148,220,7.5895,31.888,348.716785,98.324,DE
2,Eula - Weida 208,11TD8L208------9,50HERTZ,744,148,220,5.0537,19.541,264.239358,67.550,DE
3,Bentwisch - Güstrow 275,11TD8L275------7,50HERTZ,695,167,220,1.9090,12.055,158.084942,41.418,DE
4,Wuhlheide - Thyrow 291,11TD8L291------L,50HERTZ,687,6,220,2.9672,11.191,266.784,38.882,DE
...,...,...,...,...,...,...,...,...,...,...,...
2657,Y-Stokach (-Gurtweil - Laufenburg) ge-rt,11T0-0000-1626-3,TRANSNETBW,319,706,220,2.1000,16.800,347.3659,69.710,DE
2658,Y-Villingen (-Engstlatt - Kuehmoss) gn,11TD400000587-3M,TRANSNETBW,309,1427,400,0.4500,3.820,78.22566,16.200,DE
2659,Y-Weier (-Daxlanden - Kuppenheim) ge,11TD40000A574-36,TRANSNETBW,763,242,220,1.4000,10.750,230.6872,46.710,DE
2660,Y-Weinheim (-GKM - Neurott) rt,11T0-0000-1640-D,TRANSNETBW,173,1015,220,0.1000,0.200,1.57,0.120,DE


,name,zone,tso,lat,lon
0,JOACHIMOW,PL,PSE,50.760033,19.316645
1,TUILIERES,FR,RTE,44.846132,0.633183
2,GDANSK,PL,PSE,54.338682,18.709677
3,MITRY-MORY (E.D.F.),FR,RTE,48.966663,2.631343
4,Gohrpunkt,DE,Amprion GmbH,51.090932,6.709756
...,...,...,...,...,...
1492,Weiwerd,NL,TENNET NL,53.307723,6.958441
1493,JANOW,PL,PSE,51.757669,19.561529
1494,AOSTE,FR,RTE,45.583848,5.611468
1495,Barbosi,RO,TEL,45.410030,27.975590


In [18]:
edgelist = []
for index, row in df_grid.iterrows():
    edgelist.append((row.substation_1, row.substation_2))

Grid = nx.Graph(edgelist)

In [19]:
fig = go.Figure()

for index, row in df_grid.iterrows():
    df_grid.loc[index, 'start_lon'] = df_substations.loc[row.substation_1, 'lon']
    df_grid.loc[index, 'start_lat'] = df_substations.loc[row.substation_1, 'lat']
    
    df_grid.loc[index, 'end_lon'] = df_substations.loc[row.substation_2, 'lon']
    df_grid.loc[index, 'end_lat'] = df_substations.loc[row.substation_2, 'lat']
    
lons = np.empty(3 * len(df_grid))
lons[::3] = df_grid['start_lon']
lons[1::3] = df_grid['end_lon']
lons[2::3] = None
lats = np.empty(3 * len(df_grid))
lats[::3] = df_grid['start_lat']
lats[1::3] = df_grid['end_lat']
lats[2::3] = None

fig.add_trace(
    go.Scattergeo(
        locationmode = 'country names',
        lon = lons,
        lat = lats,
        mode = 'lines',
        line = dict(width = 1, color = 'red'),
        opacity = 0.5
    )
)

fig.add_trace(go.Scattergeo(
    locationmode = 'country names',
    lon = df_substations.lon,
    lat = df_substations.lat,
    hoverinfo = 'text',
    text = df_substations.name,
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

fig.show()